In [1]:
# Import Numpy & PyTorch
import numpy as np
import torch

## Linear Regression Model using PyTorch built-ins

Let's re-implement the same model using some built-in functions and classes from PyTorch.

And now using two different targets: Apples and Oranges

In [12]:
# Imports
import torch.nn as nn

In [13]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119]], dtype='float32')

In [14]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print("Inputs : \n",inputs)
print("Targets : \n",targets)

Inputs : 
 tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
Targets : 
 tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


### Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples. We'll also create a DataLoader, to split the data into batches while training. It also provides other utilities like shuffling and sampling.

In [15]:
# Import tensor dataset & data loader
from torch.utils.data import TensorDataset, DataLoader

In [16]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [17]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [18]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 87., 134.,  58.],
        [ 73.,  67.,  43.],
        [ 69.,  96.,  70.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.]])
tensor([[119., 133.],
        [ 56.,  70.],
        [103., 119.],
        [119., 133.],
        [ 22.,  37.]])


### nn.Linear
Instead of initializing the weights & biases manually, we can define the model using `nn.Linear`.

In [19]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.3974, -0.2416, -0.3264],
        [-0.0537,  0.3408,  0.4487]], requires_grad=True)
Parameter containing:
tensor([0.0378, 0.5378], requires_grad=True)


In [20]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.3974, -0.2416, -0.3264],
         [-0.0537,  0.3408,  0.4487]], requires_grad=True),
 Parameter containing:
 tensor([0.0378, 0.5378], requires_grad=True)]

In [21]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ -1.1757,  38.7439],
        [ -5.9508,  54.3563],
        [-16.6972,  67.5536],
        [ 18.1063,  26.3165],
        [-18.5849,  60.9558],
        [ -1.1757,  38.7439],
        [ -5.9508,  54.3563],
        [-16.6972,  67.5536],
        [ 18.1063,  26.3165],
        [-18.5849,  60.9558],
        [ -1.1757,  38.7439],
        [ -5.9508,  54.3563],
        [-16.6972,  67.5536],
        [ 18.1063,  26.3165],
        [-18.5849,  60.9558]], grad_fn=<AddmmBackward>)

### Loss Function
Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [22]:
# Import nn.functional
import torch.nn.functional as F

In [23]:
# Define loss function
loss_fn = F.mse_loss

In [24]:
loss = loss_fn(model(inputs) , targets)
print(loss)

tensor(5496.0347, grad_fn=<MseLossBackward>)


### Optimizer
Instead of manually manipulating the weights & biases using gradients, we can use the optimizer `optim.SGD`.

In [25]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

### Train the model

We are ready to train the model now. We can define a utility function `fit` which trains the model for a given number of epochs.

In [26]:
# Define a utility function to train the model
def fit(num_epochs, model, loss_fn, opt):
   for epoch in range(num_epochs):
       for xb,yb in train_dl:
            # Generate predictions
            pred = model(xb)
            loss = loss_fn(pred,yb)
            # Perform gradient descent
            loss.backward()
            opt.step()
            opt.zero_grad()
   print('Training loss: ', loss_fn(model(inputs), targets))

In [27]:
# Train the model for 100 epochs
fit(100 , model , loss_fn, opt)

Training loss:  tensor(30.9591, grad_fn=<MseLossBackward>)


In [28]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 59.1693,  71.1642],
        [ 79.7860,  99.4632],
        [120.8034, 134.1902],
        [ 32.4519,  41.3940],
        [ 91.0933, 114.5992],
        [ 59.1693,  71.1642],
        [ 79.7860,  99.4632],
        [120.8034, 134.1902],
        [ 32.4519,  41.3940],
        [ 91.0933, 114.5992],
        [ 59.1693,  71.1642],
        [ 79.7860,  99.4632],
        [120.8034, 134.1902],
        [ 32.4519,  41.3940],
        [ 91.0933, 114.5992]], grad_fn=<AddmmBackward>)

In [29]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])